# Illustrations


In [ ]:
%%capture
%matplotlib inline

import ContinuousFeAR as cfear
import Plot_cFeAR as plotcf
import PlotGWorld
import numpy as np; np.random.seed(0)


Save_Fig = True
Save_Fig = False

Plot_Scenario=True
# Plot_Scenario=False

# Plot_Boxes_and_Trajs = False
Plot_Boxes_and_Trajs = True

## Running Example

In [ ]:
quick_plots = False
quick_plots = True

plot_feasible_action_and_cofiguration_spaces_ = not quick_plots

arrow_width = 0.45
arrow_markersize = 10

if Plot_Scenario:
    import os 

    folder = 'FeAR_Results'

    file = 'Collision_Check_Illustration_2Agents_U_turn_2-a_num-2025-01-06_00-04-50.pkl'

    results_file_path = os.path.join(folder,file)
    print(f'{results_file_path=}')

    cfear.read_and_plot_results(results_file_path=results_file_path, plot_scenario=True,save_figs=Save_Fig,
                                plot_boxes_and_trajs=Plot_Boxes_and_Trajs, scale_scenario_size=1,
                                scale_velocities=3,
                                arrow_width=arrow_width, arrow_markersize=arrow_markersize,
                                match_scenario_lims=True,
                                plot_feasible_action_and_cofiguration_spaces_=plot_feasible_action_and_cofiguration_spaces_,
                                place_scenario_legend='extend_right_out')


## Collision check for action subset

In [ ]:
scenario_name = 'Collision_Check_Illustration_2Agents_U_turn_2'

plotcf.illustrate_subspace_collision_check(theta_step_threshold=np.pi/2, a_step_threshold=10, title='', save_fig=Save_Fig, scenario_name=scenario_name)

## Collision checks for each time interval

In [ ]:
# Hyper Parameters

NN = 20

T_ACTION=10
N_TIMESTEPS=5

A_NUM = 5
THETA_NUM = 16

A_MIN = 0
A_MAX = 1

THETA_MIN = -np.pi
THETA_MAX = np.pi

COLLISION_FREE_TRAJS = True
# COLLISION_FREE_TRAJS = False

INCLUDE_OBSTACLE = True
# INCLUDE_OBSTACLE = False

# ---------------------------------------------------------------------------- #

# Obstacles

if INCLUDE_OBSTACLE:
    obstacles = []

    obstacles.append(cfear.make_circle(x=0, y=0, radius=2))

    c_offset = 15
    lb = 5
    obstacles.append(cfear.make_rectangle(x=c_offset, y=c_offset, lx=lb, ly=lb, theta=0))
    obstacles.append(cfear.make_rectangle(x=-c_offset, y=c_offset, lx=lb, ly=lb, theta=0))
    obstacles.append(cfear.make_rectangle(x=c_offset, y=-c_offset, lx=lb, ly=lb, theta=0))
    obstacles.append(cfear.make_rectangle(x=-c_offset, y=-c_offset, lx=lb, ly=lb, theta=0))

    obstacle = cfear.MultiPolygon(obstacles).buffer(0)

else:
    obstacle=None
    
# ---------------------------------------------------------------------------- #

# Trajectories for N agents

N = NN
l = 1
t_action=T_ACTION
n_timesteps=N_TIMESTEPS



x0_mean = 0
x0_std = 10

y0_mean = 0
y0_std = 10

v0x_mean = 0
v0x_std = 1

v0y_mean = 0
v0y_std = 1

a_mean = 0
a_std = 0.2

theta_mean = 0
theta_std = np.pi/2

# ----------------------------------------------------------------------------------------  #

x0, y0 = cfear.generate_starting_locations(x0_mean=x0_mean, x0_std=x0_std, obstacle=obstacle,
                                     y0_mean=y0_mean, y0_std=y0_std,
                                     n=N, l=2*l)
v0x = v0x_mean + v0x_std * np.random.randn(N, 1)
v0y = v0y_mean + v0y_std * np.random.randn(N, 1)
a = a_mean + a_std * np.random.randn(N, 1)
theta = theta_mean + theta_std * np.random.randn(N, 1)

# ----------------------------------------------------------------------------------------  #

x_, y_, t_ = cfear.make_trajectories(x0=x0,y0=y0,v0x=v0x,v0y=v0y,a=a, theta=theta,
                                 t_action=t_action, n_timesteps=n_timesteps)
trajs_boxes = cfear.get_boxes_for_trajs(x_, y_, lx=l, ly=l)
trajs_hulls = cfear.get_trajs_hulls(trajs_boxes)

# ----------------------------------------------------------------------------------------  #

In [ ]:
if COLLISION_FREE_TRAJS:
    trajs_hulls, trajs_boxes = cfear.get_collision_free_trajs(trajs_hulls, trajs_boxes, obstacle=obstacle,
                                                              plot_colliding_timesteps=True,
                                                              save_plot_colliding_timesteps=Save_Fig)
